# Export CD34+ Leukemia Cell Calls

#### This notebook combines the CD34+ leukemia cell calling from each patient in one list for export

In [1]:
import json
import numpy as np
from PIL import  Image
import io
import base64
import zlib
import cv2
from string import Template
import glob
import os
import tifffile
import math
from skimage import data, segmentation, util, measure
import pandas as pd
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt

### Read in size and RBC filtered cell list

In [2]:
filt_cells = pd.read_csv('/data/Zhaolab/1_AMLCosMx/Final_scripts/4_CellTyping/2_FilterbySize/Cells_filtbyRBCandsize.csv', index_col=0)
filt_cells

,label,area,y,x,Patient,FOV,RBC
ID,,,,,,,
P51_FOV01_cell_1,1,903,15.766334,74.070875,P51,1,0
P51_FOV01_cell_2,2,1122,14.835116,117.772727,P51,1,0
P51_FOV01_cell_3,3,2034,20.265487,218.211898,P51,1,0
P51_FOV01_cell_4,4,2534,41.141279,962.670481,P51,1,0
P51_FOV01_cell_5,5,1006,16.085487,1059.072565,P51,1,0
...,...,...,...,...,...,...,...
P58_FOV23_cell_7966,7966,731,3638.218878,4327.127223,P58,23,0
P58_FOV23_cell_7971,7971,596,3640.771812,4117.850671,P58,23,0
P58_FOV23_cell_7972,7972,359,3641.122563,4992.493036,P58,23,0


In [3]:
# load FOV metadata
FOV_metadata = pd.read_csv('/data/Zhaolab/1_AMLCosMx/Final_scripts/4_CellTyping/FOV_metadata.csv', index_col=0)
FOV_metadata.loc['P57_FOV17'] = ['C', 'CR'] # add row for P57 FOV17

filt_cells['Timepoint'] = ''
for i in range(filt_cells.shape[0]):
    sample = filt_cells.iloc[i].name[0:9]
    filt_cells['Timepoint'].iloc[i] = FOV_metadata['Timepoint'].loc[sample]

/tmp/ipykernel_1931532/1523178950.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_cells['Timepoint'].iloc[i] = FOV_metadata['Timepoint'].loc[sample]


In [4]:
filt_cells

,label,area,y,x,Patient,FOV,RBC,Timepoint
ID,,,,,,,,
P51_FOV01_cell_1,1,903,15.766334,74.070875,P51,1,0,A
P51_FOV01_cell_2,2,1122,14.835116,117.772727,P51,1,0,A
P51_FOV01_cell_3,3,2034,20.265487,218.211898,P51,1,0,A
P51_FOV01_cell_4,4,2534,41.141279,962.670481,P51,1,0,A
P51_FOV01_cell_5,5,1006,16.085487,1059.072565,P51,1,0,A
...,...,...,...,...,...,...,...,...
P58_FOV23_cell_7966,7966,731,3638.218878,4327.127223,P58,23,0,C
P58_FOV23_cell_7971,7971,596,3640.771812,4117.850671,P58,23,0,C
P58_FOV23_cell_7972,7972,359,3641.122563,4992.493036,P58,23,0,C


### Read in leukemia cell calls

In [5]:
patients = ['P51_backsub', 'P52', 'P53', 'P56', 'P57', 'P58']

In [7]:
all_pts_dfs = []

for patient in patients:

    # read in Lk cell labels from each patient
    onept_LKcells = pd.read_csv('/data/Zhaolab/1_AMLCosMx/Final_scripts/4_CellTyping/3_IdentifyLKcells/' + patient + '/' + patient + '_LkCells.csv')

    # give each cell unique ID, set as index
    onept_LKcells['ID'] = onept_LKcells['Patient'] + '_FOV' + onept_LKcells['FOV'].astype(str).str.zfill(2) + '_cell_' + onept_LKcells['cell_ID'].astype(str)
    onept_LKcells = onept_LKcells.set_index('ID')

    # add column to identify all cells as Lk cells
    onept_LKcells['Lk_cell'] = 1

    # save single patient output to list
    all_pts_dfs.append(onept_LKcells)

In [8]:
# concatenate into one table for all patients
all_pt_data = pd.concat(all_pts_dfs)
all_pt_data

,FOV,cell_ID,Patient,Lk_cell
ID,,,,
P51_FOV01_cell_12,1,12,P51,1
P51_FOV01_cell_34,1,34,P51,1
P51_FOV01_cell_45,1,45,P51,1
P51_FOV01_cell_231,1,231,P51,1
P51_FOV01_cell_267,1,267,P51,1
...,...,...,...,...
P58_FOV23_cell_4090,23,4090,P58,1
P58_FOV23_cell_5074,23,5074,P58,1
P58_FOV23_cell_6205,23,6205,P58,1


In [9]:
# load FOV metadata
FOV_metadata = pd.read_csv('/data/Zhaolab/1_AMLCosMx/Final_scripts/4_CellTyping/FOV_metadata.csv', index_col=0)
FOV_metadata.loc['P57_FOV17'] = ['C', 'CR'] # add row for P57 FOV17

all_pt_data['Timepoint'] = ''
for i in range(all_pt_data.shape[0]):
    sample = all_pt_data.iloc[i].name[0:9]
    all_pt_data['Timepoint'].iloc[i] = FOV_metadata['Timepoint'].loc[sample]

/tmp/ipykernel_1931532/2296965825.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_pt_data['Timepoint'].iloc[i] = FOV_metadata['Timepoint'].loc[sample]


In [10]:
all_pt_data

,FOV,cell_ID,Patient,Lk_cell,Timepoint
ID,,,,,
P51_FOV01_cell_12,1,12,P51,1,A
P51_FOV01_cell_34,1,34,P51,1,A
P51_FOV01_cell_45,1,45,P51,1,A
P51_FOV01_cell_231,1,231,P51,1,A
P51_FOV01_cell_267,1,267,P51,1,A
...,...,...,...,...,...
P58_FOV23_cell_4090,23,4090,P58,1,C
P58_FOV23_cell_5074,23,5074,P58,1,C
P58_FOV23_cell_6205,23,6205,P58,1,C


In [13]:
patients = ['P51', 'P52', 'P53', 'P56', 'P57', 'P58']

In [14]:
# check percentage of each patient's total cells are identified as leukemia cells (at baseline/timepoint A)
all_cells_A = filt_cells[filt_cells['Timepoint'] == 'A']
lk_cells_A = all_pt_data[all_pt_data['Timepoint'] == 'A']

for patient in patients:
    print(patient + ': ' + str(lk_cells_A[lk_cells_A['Patient'] == patient].shape[0] / all_cells_A[all_cells_A['Patient'] == patient].shape[0] * 100))


P51: 5.4958279845956355
P52: 10.33701732842134
P53: 6.032981665471679
P56: 2.3726388093875213
P57: 3.201934349355217
P58: 1.9520025366124332


In [15]:
# save list of all cell IDs identified as leukemia cells
all_pt_data.to_csv('LeukemiaCells_V8_3-7-24.csv')